In [31]:
#This cell handles installs if needed, fixes environment for python ffmpeg if needed, and imports all needed packages.

#!pip install -U openai --upgrade
#!pip install openai-whisper
#!pip install ffmpeg-python
import os
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
import gradio as gr
import openai
from openai import OpenAI
import json
import csv
import pandas as pd
import whisper
import ffmpeg
import random
import glob
client = OpenAI()

new_directory = "C:\\ffmpeg\\bin"

# Get the current PATH environment variable
current_path = os.environ.get('PATH', '')

# Check if the directory is already in PATH
if new_directory not in current_path:
    # Add the new directory to PATH
    updated_path = current_path + ";" + new_directory
    os.environ['PATH'] = updated_path
    print("Updated PATH:", updated_path)
else:
    print("Directory already in PATH.")



Directory already in PATH.


In [38]:
#this cell creates all the helper functions needed.

openai.api_key=os.environ['OPENAI_API_KEY']


###################################################
#The section below is for general gpt responses   #
###################################################
def get_ai_reply(message, model="gpt-3.5-turbo-16k", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

def chat(message, audio, chatbot_messages, history_state):
    # Initialize chatbot_messages and history_state if they are not provided
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    if audio:
        message = preprocess_audio(audio)
    # Try to get the AI's reply using the get_ai_reply function
    try:
        prompt = """
        You are an AI assistant for a medical call center. The center's function is to take, record call information, and assist callers.
        
        ##Analyze the transcribed audio and gather then following information:
        - Name of Patient
        - Name of Caller
        - Identity of Caller (Patient, Care-giver, Insurance Representative, Other Medical Office, Other)
        - Reason for Calling
        - Appointment Requested (True or False, or N/A)
        - Appointment Status (True or False or N/A)
            - Reason for Failure
        - Problems/issues Raised by The Caller
        - Sentiment of Call (Positive, Negative, or Neutral)
        """
        prompt2 = """
        You are a helpful JSON creator.

        Take the input and output it into JSON using the following keys and types:
        - Name of Patient [string]
        - Name of Caller [string]
        - Identity of Caller [string]
        - Reason for Calling [string]
        - Appointment Requested [bool or null]
        - Appointment Status [bool or null]
        - Reason for Failure [string or null]
        - Problems/issues Raised [string or null]
        - Sentiment of Call [string]
        """
        ai_reply = get_ai_reply(message, model="gpt-3.5-turbo-16k", system_message=prompt.strip(), message_history=history_state)
            
        # Append the user's message and the AI's reply to the history_state list
        history_state.append({"role": "user", "content": message})
        history_state.append({"role": "assistant", "content": ai_reply})
        
        ai_reply = get_ai_reply(ai_reply, model="gpt-3.5-turbo-16k", system_message=prompt2.strip(), message_history=history_state)
        history_state.append({"role": "assistant", "content": ai_reply})
        
        # Append the user's message and the AI's reply to the chatbot_messages list for the UI
        chatbot_messages.append((message, ai_reply))

        # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    return None, None, chatbot_messages, history_state

def stats(message):
    csv_file_path = 'data.csv'            
    directory = 'transcripts'
    all_json_data = []
    
    # Try to get the AI's reply using the get_ai_reply function
    try:
        prompt = """
        You are an AI assistant for a medical call center. The center's function is to take, record call information, and assist callers.
                
        ##Rules
        Don't exlcude any information, even if N/A.
        
        ##Analyze the transcribed audio and gather the following information:
        - name_of_patient
        - name_of_caller (patient or person on behalf of the patient.)
        - identity_of_caller [patient, patient_care_giver, insurance_representative, other_medical_office, other]
        - reason_for_calling [make_appointment, insurance_verification, change_appointment, cancel_appointment, request_test_results, request_perscription_refill, request_referral, inquiry_about_services, request_medical_records, provide_feedback, other]
        - appointment_scheduled [true, false, N/A] (True if appointment is successfully scheduled for a date, false if appointment cancelled or failed to schdule, N/A if not appilicable)
        - reason_for_schdule_failure [N/A, no_availability, no_referral, not_a_service, insurance_converage, incomplete_inforamtion, other]
        - problems_raised_by_caller (if no problem: N/A)
        - sentiment_of_call [positive, negative, neutral]
        """
        prompt2 = """
        You are a helpful JSON creator.

        Take the input and output it into JSON using the following keys and types:
        - name_of_patient [string]
        - name_of_caller [string]
        - identity_of_caller [string]
        - reason_for_calling [string]
        - appointment_scheduled [bool or null]
        - reason_for_schedule_failure [string or null]
        - problems_raised [string or null]
        - sentiment_of_call [string]
        """

        # Loop through each file in the directory
        for filename in os.listdir(directory):
            if filename.endswith(".txt"):
                file_path = os.path.join(directory, filename)
                with open(file_path, 'r') as file:
                    file_content = file.read()
            
                    ai_reply = get_ai_reply(file_content, model="gpt-3.5-turbo-16k", system_message=prompt.strip())

                    # Append the user's message and the AI's reply to the history_state list
                    #history_state.append({"role": "user", "content": inputs[i]})
                    #history_state.append({"role": "assistant", "content": ai_reply})

                    ai_reply = get_ai_reply(ai_reply, model="gpt-3.5-turbo-16k", system_message=prompt2.strip())
                    #history_state.append({"role": "assistant", "content": ai_reply})

                    ##Add entry to CSV
                    if(is_valid_json(ai_reply)):
                        json_data = json.loads(ai_reply)
                        all_json_data.append(json_data)
        #data = [ { "name_of_patient": "Mark Johnson", "name_of_caller": "Mark Johnson", "identity_of_caller": "patient", "reason_for_calling": "provide_feedback", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "long wait time, doctor seemed rushed and didn't listen to concerns", "sentiment_of_call": "negative" }, { "name_of_patient": "Sarah Johnson", "name_of_caller": "Sarah Johnson", "identity_of_caller": "patient", "reason_for_calling": "inquiry_about_services", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "John Smith", "name_of_caller": "John Smith", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_referral", "problems_raised": "", "sentiment_of_call": "neutral" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "change_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "frustration with lack of response and inconvenience caused", "sentiment_of_call": "negative" }, { "name_of_patient": "John Smith", "name_of_caller": "Lisa", "identity_of_caller": "insurance_representative", "reason_for_calling": "insurance_verification, discuss_billing_process", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "Emma Thompson", "name_of_caller": "Mrs. Thompson", "identity_of_caller": "patient_care_giver", "reason_for_calling": "make_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "", "name_of_caller": "", "identity_of_caller": "", "reason_for_calling": "inquiry_about_services", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "Mark Johnson", "name_of_caller": "Mark Johnson", "identity_of_caller": "patient", "reason_for_calling": "provide_feedback", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "long wait time, doctor seemed rushed and didn't listen to concerns", "sentiment_of_call": "negative" }, { "name_of_patient": "Sarah Johnson", "name_of_caller": "Sarah Johnson", "identity_of_caller": "patient", "reason_for_calling": "inquiry_about_services", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "John Smith", "name_of_caller": "John Smith", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_referral", "problems_raised": "", "sentiment_of_call": "neutral" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "change_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "frustration with lack of response and inconvenience caused", "sentiment_of_call": "negative" }, { "name_of_patient": "John Smith", "name_of_caller": "Lisa", "identity_of_caller": "insurance_representative", "reason_for_calling": "insurance_verification, discuss_billing_process", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "Emma Thompson", "name_of_caller": "Mrs. Thompson", "identity_of_caller": "patient_care_giver", "reason_for_calling": "make_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "inquiry_about_services", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "not_a_service", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "neutral" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "neutral" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "neutral" } ]
        raw = pd.DataFrame(data)
        raw.to_csv()
        results = EXTRACT_INFO(data)
        df = pd.DataFrame([results])
        
        num_entries = f"<h1>num_entries:</h1>\n<h2>{df['num_entries'].values[0]}</h2>"
        total_pos = f"<h1>total_positives:</h1>\n<h2>{df['total_pos'].values[0]}</h2>"
        total_neg = f"<h1>total_negatives:</h1>\n<h2>{df['total_neg'].values[0]}</h2>"
        total_neu = f"<h1>total_nerutrals</h1>\n<h2>{df['total_neu'].values[0]}</h2>"

        # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    return num_entries, total_pos, total_neg, total_neu, raw, csv_file_path


#######################################################
#The section below is for generation of conversations #
#######################################################

def generate_ai_chat(message, prompt, model="gpt-3.5-turbo-16k", temperature=0.3):

    # Create the message list with the system's refinement prompt and the user's transcribed audio
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": message}
    ]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    print(completion.choices[0].message.content)
    print(completion.choices[0].message.content.strip())
    return completion.choices[0].message.content.strip()
def current_output_num():
    path = os.getcwd()

    # List all 'output*.mp3' files
    files = glob.glob(os.path.join(path, 'output*.mp3'))

    # Extract numbers and find the highest one
    numbers = [int(os.path.splitext(os.path.basename(file))[0].replace('output', '')) for file in files]
    latest_number = max(numbers) if numbers else 0

    # The next number to use
    next_number = latest_number + 1
    return next_number

def generate_chat(number):
    print("Generating a synthetic conversation...\n")
    identity = ["patient", "care giver", "insurance representative", "another medical office"]
    callreason = ["make an appointment", "change appointment", "cancel appointment"]
    scheduled = ["successful appointment scheduled", "unsuccessful in scheduling appointment"]
    number = int(number)
    for x in range(number):
        ident = random.choice(identity)
        sched = random.choice(scheduled)
        reason = random.choice(callreason)
        prompt = """
            ## Overview
                You are an AI assistant meant to generate entire transcripts for training data in machine learning tasks. You are not in a role-playing scenario where you give a line and wait for another line. You control both sides of the conversation and must output the entire thing. 
  
            ## Context
                You will generate a full script for a phone call to a call center for a health system called RGV Health.RGV Health is part of a minority serving health system. Many phone calls represent issues that plauge these minority communities.
            ## Task
                You are creating training data based on full phone conversations. Your data will be converted to spoken word and used to help train AI models and humans. Be sure to follow the guidelines in order to provide a smooth collection of data. 

            ## Guidelines for Synthetic Transcript Generation
                The phone call should include the following information naturally:
                    - name of patient
                    - name of caller (if different than patient such as calling for a child)
                    - identity of caller {ident0}
                    - reason for calling: {reason0}
                    - appointment scheduled: {sched0}
                    - if appointment is not able to be scheduled, give a valid reasoning
                The phone call should only include spoken words and no tags such as [pauses] [holds] [hold music] or [end of conversation]. Since this will be turned to speech, such tags will hinder data collection. 
    
                You should generate both the caller and the agent taking the phone call. Your goal is to output the entire synthetic conversation. 

            ## Output
                Your output should follow a structured format:
                    -Either the caller or answering agent can begin the conversation, but it should not be indicated which is speaking.
                    -The entire conversation should be in a single output. 
                    -The entire conversation can be supplied within strings to separate speakers.
                Example:
                "Hello, I would like to schedule an appointment for next week."
                "Sure, we at RGV Health would be happy to help make an appointment for you. Can I get your name and date of birth?"
                "My name is Ken and my date of birth is Janurary third, 1970."
    
                Bad Example:
                "Hello, this is RGV Heath", then waiting for a response.
        """.format(
        ident0=ident,
        reason0=reason,
        sched0=sched
        )
        text = generate_ai_chat("generate the whole conversation", prompt)
        text = text.replace("Caller: ", "")
        text = text.replace("Agent: ", "")
        response = OpenAI().audio.speech.create(
          model="tts-1",
          voice="alloy",
          input=text
        )
        num = current_output_num()
        response.stream_to_file(f"output{num}.mp3")
        print(f"output{num}.mp3 created.")



##########################################################
#The section below is for transcription of conversations #
##########################################################

def transcribe_audio(audio):
    model = whisper.load_model("base")
    transcription = ""
    # Load and transcribe the audio file
    result = model.transcribe(audio)
    for segment in result["segments"]:
        transcription += f"{segment['text']}\n"
    return transcription

def refine_audio(transcribed_audio, refinement_prompt, model="gpt-3.5-turbo-16k", temperature=0):

    # Create the message list with the system's refinement prompt and the user's transcribed audio
    messages = [
        {"role": "system", "content": refinement_prompt},
        {"role": "user", "content": transcribed_audio}
    ]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Check if the completion object contains the expected data
    if completion.choices and completion.choices[0].message:
        # Extract and return the AI's response from the API response
        return completion.choices[0].message.content.strip()
    
    return ""

def preprocess_audio(audio):
    transcription = transcribe_audio(audio)
    prompt1 = """
    Your task is to clean up the given text.
    The text transcribes a conversation between two people. 
    Separate the two conversations by including quotation marks around each person's words.
    --Example--
    "Hi, Frank, how are you doing?" "I'm doing well, how about you?" "I'm doing just fine as well."
    """
    transcription2 = refine_audio(transcription, prompt1.strip())
    print(transcription2)
    prompt2 = """
    You are an editor assisting a scriptwriter. Your task is to format the given conversation into a readable script for two characters.
    One of the characters is Agent. The other should be introduced in the script and used appropriately. 
    Each line of dialogue must be preceded by the speaking character's name followed by a colon.
    --Example--
    Agent: "Hi, Frank, how are you doing?"
    Frank: "I'm doing well, ow about you?"
    Agent: "I'm doing just fine as well. How can I help you today?"
    Frank: "Well, I want to try and set up an appointment."
    Agent: "Sure, I can help with that."
    ----
    Instructions:

    Identify the two speakers in the conversation.
    Label each line of dialogue with the appropriate speaker's name.
    Maintain the original content and context of the conversation.
    Format the dialogue in a clear, script-style presentation.
    
    Failure to format the conversation correctly with named labels for each speaker will result in a significant misrepresentation of the dialogue, affecting the script's quality and clarity.
    
    """

    transcription3 = refine_audio(transcription2, prompt2.strip())
    print(transcription3)
    return transcription3

def transcribe_folder():
    directory = os.getcwd()
    for filename in os.listdir(directory):
        if filename.endswith(".mp3"):
            file_path = os.path.join(directory, filename)
            chat("", file_path, None, None)

################################################################
#The functions below are to help with data analysis and output #
################################################################

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def is_valid_json(json_str):
    try:
        json.loads(json_str)
        return True
    except ValueError:
        return False
    
def create_csv_file(file_path, data):
    # Check if the CSV file already exists
    file_exists = os.path.isfile(file_path)

    # Open the CSV file in append mode
    with open(file_path, 'a', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=data.keys())

        # Write the header row if the file is newly created
        if not file_exists:
            writer.writeheader()

        # Write the data row
        writer.writerow(data)
        
def EXTRACT_INFO(data):
    # Initialize variables
    num_entries = len(data)
    total_pos = 0
    total_neg = 0
    total_neu = 0
    make_appointment_count = 0
    appointment_scheduled_count = 0
    make_appointment_failure_count = 0
    positive_count = 0
    neutral_count = 0
    negative_count = 0
    schedule_failure_reasons = {}

    # Analyze the data
    for entry in data:
        sentiment = entry['sentiment_of_call']
        reason = entry['reason_for_calling']
        appointment_scheduled = entry['appointment_scheduled']
        reason_for_schedule_failure = entry['reason_for_schedule_failure']

        if sentiment == 'positive':
            total_pos += 1
        elif sentiment == 'negative':
            total_neg += 1
        elif sentiment == 'neutral':
            total_neu += 1

        if reason == 'make_appointment':
            make_appointment_count += 1
            if sentiment == 'positive':
                positive_count += 1
            elif sentiment == 'neutral':
                neutral_count += 1
            elif sentiment == 'negative':
                negative_count += 1

            if appointment_scheduled == "FALSE":
                make_appointment_failure_count += 1
                if reason_for_schedule_failure in schedule_failure_reasons:
                    schedule_failure_reasons[reason_for_schedule_failure] += 1
                else:
                    schedule_failure_reasons[reason_for_schedule_failure] = 1

        if appointment_scheduled == "TRUE":
            appointment_scheduled_count += 1

    # Calculate percentages
    appointment_scheduled_percentage = (appointment_scheduled_count / make_appointment_count) * 100
    positive_percentage = (positive_count / make_appointment_count) * 100
    neutral_percentage = (neutral_count / make_appointment_count) * 100
    negative_percentage = (negative_count / make_appointment_count) * 100

#     # Print the results
#     print("Number of entries:", num_entries)
#     print("Number of 'positive' sentiments:", total_pos)
#     print("Number of 'negative' sentiments:", total_neg)
#     print("Number of 'neutral' sentiments:", total_neu)

#     print("\nNumber of times 'make_appointment' was the reason for calling:", make_appointment_count)
#     print("Number of times 'appointment_scheduled' was True:", appointment_scheduled_count)
#     print("Percentage of appointments scheduled: {:.2f}%".format(appointment_scheduled_percentage))

#     print("\nNumber of 'make_appointment' entries:", make_appointment_count)
#     print("Number of 'positive' sentiments specific to 'make_appointment':", positive_count)
#     print("Number of 'neutral' sentiments specific to 'make_appointment':", neutral_count)
#     print("Number of 'negative' sentiments specific to 'make_appointment':", negative_count)
#     print("Percentage of 'positive' sentiments specific to 'make_appointment': {:.2f}%".format(positive_percentage))
#     print("Percentage of 'neutral' sentiments specific to 'make_appointment': {:.2f}%".format(neutral_percentage))
#     print("Percentage of 'negative' sentiments specific to 'make_appointment': {:.2f}%".format(negative_percentage))

#     print("\nNumber of 'make_appointment' entries that resulted in 'appointment_scheduled: FALSE':", make_appointment_failure_count)
    for reason, count in schedule_failure_reasons.items():
        percentage = (count / make_appointment_failure_count) * 100
#         print("Reason for schedule failure:", reason)
#         print("Percentage: {:.2f}%".format(percentage))
        
    results = {
        "num_entries": num_entries,
        "total_pos": total_pos,
        "total_neg": total_neg,
        "total_neu": total_neu,
        "make_appointment_count": make_appointment_count,
        "appointment_scheduled_count": appointment_scheduled_count,
        "appointment_scheduled_percentage": appointment_scheduled_percentage,
        "positive_count": positive_count,
        "neutral_count": neutral_count,
        "negative_count": negative_count,
        "positive_percentage": positive_percentage,
        "neutral_percentage": neutral_percentage,
        "negative_percentage": negative_percentage,
        "make_appointment_failure_count": make_appointment_failure_count,
        "schedule_failure_reasons": schedule_failure_reasons
    }
    
    table1 = {
        "num_entries": results["num_entries"],
        "total_pos": results["total_pos"],
        "total_neg": results["total_neg"],
        "total_neu": results["total_neu"]
    }

    # Create a dictionary for the second table
    table2 = {
        "make_appointment_count": results["make_appointment_count"],
        "appointment_scheduled_count": results["appointment_scheduled_count"],
        "appointment_scheduled_percentage": results["appointment_scheduled_percentage"]
    }

    # Return the results
    return results

In [40]:
#This cell creates the gradio app in which everything is updated.

def gradioGUI():
    with gr.Blocks() as app:
        with gr.Tab("Generate Audio"):
            num = [gr.inputs.Dropdown(list(range(1,20)), label="Number of Conversations to Create")]
            btn1 = gr.Button(value="Generate")
            btn1.click(generate_chat, inputs=num)

        with gr.Tab("Process Single Audio"):
            chatbot = gr.Chatbot(label="Conversation")
            message = gr.Textbox(label="Message")
            audio = gr.inputs.Audio(source="upload", type="filepath")
            history_state = gr.State()
            btn2 = gr.Button(value="Analyze")
            btn2.click(chat, inputs=[message, audio, chatbot, history_state], outputs=[message, audio, chatbot, history_state])

        with gr.Tab("Process Audios in Folder"):
            chatbot = gr.Chatbot(label="Conversation")
            audio = gr.inputs.Audio(source="upload", type="filepath")
            history_state = gr.State()
            btn = gr.Button(value="Transcribe chats in folder")
            btn.click(transcribe_folder)

        with gr.Tab("Stats"):
            message = gr.Textbox(label="Message")
            num_entries = gr.HTML()
            total_pos = gr.HTML()
            total_neg = gr.HTML()
            total_neu = gr.HTML()
            csv = gr.Dataframe()
            csv_file = gr.File(label="Download (CSV)", interactive=False)
            btn = gr.Button(value="Send")
            btn.click(stats, inputs=[message], outputs=[num_entries, total_pos, total_neg, total_neu, csv, csv_file])

    return app

app = gradioGUI()
app.queue()  # this is to be able to queue multiple requests at once
app.launch(share=True)

C:\Users\Ethen\AppData\Local\Temp\ipykernel_1668\566981138.py:6: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  num = [gr.inputs.Dropdown(list(range(1,20)), label="Number of Conversations to Create")]
C:\Users\Ethen\AppData\Local\Temp\ipykernel_1668\566981138.py:6: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  num = [gr.inputs.Dropdown(list(range(1,20)), label="Number of Conversations to Create")]
C:\Users\Ethen\AppData\Local\Temp\ipykernel_1668\566981138.py:13: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  audio = gr.inputs.Audio(source="upload", type="filepath")
C:\Users\Ethen\AppData\Local\Temp\ipykernel_1668\566981138.py:13: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  audio = gr.inp

Running on local URL:  http://127.0.0.1:7867
Running on public URL: https://07112f7352a420e035.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


C:\Users\Ethen\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


"Hello, I would like to schedule an appointment for next week."
"Sure, we at RGV Health would be happy to help make an appointment for you."
"Can I get your name and date of birth?"
"My name is Sarah Johnson and my date of birth is June 15, 1985."
"Thank you, Sarah."
"What is the reason for your appointment?"
"I need to see a dermatologist."
"I've been having some skin issues lately."
"I'm sorry to hear that."
"We have a few dermatologists available next week."
"Are you available on Monday or Tuesday?"
"Monday works best for me."
"Great."
"We have an opening at 10 a.m. on Monday."
"Does that work for you?"
"Yes, that works for me."
"Perfect."
"I have scheduled your appointment with Dr. Smith on Monday June 21 at 10 a.m."
"Is there anything else I can assist you with?"
"No, that's all."
"Thank you for your help."
"You're welcome, Sarah."
"If you have any further questions, feel free to give us a call."
"Have a great day."
"Thank you."
"You too. Goodbye."
"Goodbye."
Agent: "Hello, I woul